# Практическое занятие № 3

## Обнуражение и классификация объектов

Задачи обнаружения и классификации будем решать при помощи [YOLOv8](https://github.com/ultralytics/ultralytics)

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

Обнаружение объектов

In [ ]:
model = YOLO("yolov8n.pt")

img = cv2.imread('cats.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = model(img)

for r in results:
    res_plotted = r.plot()

    plt.imshow(res_plotted)
    plt.show()

Классификация объектов

In [ ]:
model = YOLO("yolov8n-cls.pt")

img = cv2.imread('cats.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = model(img)

Сегментация объектов

In [ ]:
model = YOLO("yolov8n-seg.pt")

img = cv2.imread('cats.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = model(img)

for r in results:
    res_plotted = r.plot()

    plt.imshow(res_plotted)
    plt.show()

### Как обучить детектор самому

В этом деле очень может помочь [Roboflow](https://universe.roboflow.com/)

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="key")
project = rf.workspace("220616nishikura").project("catdetector")
dataset = project.version(2).download("yolov8")

In [ ]:
model = YOLO("yolov8n.pt", 'detect')

dataset = 'path/to/dataset/file.yaml'
model.train(data=dataset, epochs=100, imgsz=320, device='cuda:0', batch=8)

Есть очень хорошая библиотека для решения задач классификации - [pytorch-image-models](https://huggingface.co/docs/timm/index)

Ссылка на [github](https://github.com/huggingface/pytorch-image-models)

Рассмотрим простенький пример на классификацию

In [ ]:
import requests
from PIL import Image
from io import BytesIO
url = 'https://datasets-server.huggingface.co/assets/imagenet-1k/--/default/test/12/image/image.jpg'
image = Image.open(requests.get(url, stream=True).raw)
image

In [ ]:
model = timm.create_model('mobilenetv3_large_100', pretrained=True).eval()
transform = timm.data.create_transform(
    **timm.data.resolve_data_config(model.pretrained_cfg)
)

In [ ]:
image_tensor = transform(image)
image_tensor.shape

In [ ]:
output = model(image_tensor.unsqueeze(0))
output.shape

In [ ]:
probabilities = torch.nn.functional.softmax(output[0], dim=0)
probabilities.shape

In [ ]:
values, indices = torch.topk(probabilities, 5)
indices

In [ ]:
IMAGENET_1k_URL = 'https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt'
IMAGENET_1k_LABELS = requests.get(IMAGENET_1k_URL).text.strip().split('\n')
[{'label': IMAGENET_1k_LABELS[idx], 'value': val.item()} for val, idx in zip(values, indices)]

## Распознавание лиц

Для решение задачи распознавания лиц будем использовать [facenet-pytorch](https://github.com/timesler/facenet-pytorch)

In [ ]:
import torch

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1

mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [ ]:
image_path = 'path/to/image.png'
img = Image.open(image path)

In [ ]:
aligned = mtcnn(img)
aligned

In [ ]:
import lovely_tensors as lt

In [ ]:
emb = resnet(aligned.unsqueeze(0))
lt.lovely(emb)

Добавим еще одного человека, и извлечем эмбеденги

И посчитаем схожесть между эмбеденгами

In [1]:
from sklearn import preprocessing
import numpy as np

In [ ]:
def calc_distance(embedding1, embedding2):
    emb1 = preprocessing.normalize(np.reshape(embedding1, (-1, 2)))
    emb2 = preprocessing.normalize(np.reshape(embedding2, (-1, 2)))
    diff = np.subtract(emb1, emb2)
    dist = np.sum(np.square(diff))
    return dist

Считаем дистанцию между эмбедингами

Попробуем заимплементировать в наш пайплайн другой детектор, например [yolo](https://github.com/elyha7/yoloface)